In [1]:
from typing import List
import torch
import numpy as np
from torch import LongTensor, Tensor

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import queue

class BatchQueue:
    dims_shape = (3,)
    def __init__(self, dims: Tensor):
        """
        dims: [3] := (L,B,F), L>=1, B>=1, F>=1
        """

        self._q = queue.Queue()


    def dequeue(self, batch_indices: LongTensor) -> Tensor:
        """
        batch_indices: [N], 1<=N<=B
        Can assume there's at least one element at each batch index
        returns: [N,F]
        """
        pass


    def enqueue(self, values: Tensor, batch_indices: LongTensor) -> None:
        """
        values: [T,N,F], 1<=T<=L, 1<=N<=B, F>=1
        batch_indices: [N]
        Can assume there's space left in each of index
        """

        pass

    def peek(self, location: List[str], batch_indices: LongTensor) -> Tensor:
        """
        location: [N], 1<=N<=B
        batch_indices: [N]
        Can return any value if the queue is empty
        returns: [N,F]
        """
        pass

In [7]:
def sanity_check():
    ### Keep in mind that these checks are only basic helpers and do not cover all cases. ###
    # TODO: change this path to match the mel_spectograms path in the drive
    mel_spectrograms_filepath = "mel_spectrograms.npy"
    with open(
        mel_spectrograms_filepath,
        "rb",
    ) as f:
        mels = torch.tensor(np.load(f))

    expected_results = [
        torch.cat([
            mels[0, 0, :].unsqueeze(0),
            mels[4, 1, :].unsqueeze(0),
            torch.zeros([1,240000]),
        ]),
        torch.cat([
            mels[0, 0, :].unsqueeze(0),
            mels[0, 3, :].unsqueeze(0),
        ]),
        torch.cat([
            mels[1, 0, :].unsqueeze(0),
            mels[1, 3, :].unsqueeze(0),
            mels[4, 2, :].unsqueeze(0),
        ]),
        mels[4, 0, :].unsqueeze(0),
    ]
    results = []

    q = BatchQueue(torch.Tensor([10, 5, 240000]))
    q.enqueue(mels, batch_indices=torch.tensor([0, 1, 2, 4]))
    results.append(q.peek(['head', 'tail', 'head'], batch_indices=torch.tensor([0, 1, 3])))
    results.append(q.dequeue(batch_indices=torch.tensor([0, 4])))
    results.append(q.peek(['head', 'head', 'tail'], batch_indices=torch.tensor([0, 4, 2])))
    results.append(q.peek(['tail'], batch_indices=torch.tensor([0])))

    assert all([(res == expected_res).all() for res,expected_res in zip(results, expected_results)]), "Wrong results"

In [14]:
sanity_check()

tensor([1.0000e+01, 5.0000e+00, 2.4000e+05])


AttributeError: 'bool' object has no attribute 'all'